# Anwendung von maschinellem Lernen auf den KHK_Klassifikation.csv Datensatz

## Praktische Demonstration für verschiedene machine Learning Modelle

### Tim Bleicher, Linus Pfeifer

Dieses Jupyter Notebook demonstriert die Anwendung von verschiedenen Machine Learning Modellen auf den KHK_Klassifikation.csv Datensatz. 

**Inhaltsverzeichnis:**
TODO

## 1. Einbindung der Daten

Zu beginn des Projekts werden die Daten zunächst geladen um diese im anschluss analysieren und nutzen zu können.

In [ ]:
pip install -r requirements.txt

In [17]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [2]:
data = pd.read_csv('KHK_Klassifikation.csv', sep=',')

In [ ]:
print(data.head())

## 1.1 explorative Analyse der Daten 
TODO

## 2. PCA-Dimensionsreduzierung zur Visualisierung und Analyse der Daten 

### Funktionsweise von PCA
Die Hauptkomponentenanalyse (PCA) dient der Dimensionsreduktion eines Datensatzes. Dies ermöglicht beispielsweise verschiedene Analyse des gesamten Datensatzes (mit mehr als 3 Dimensionen), wobei die Ergebnisse durch die Dimensionsreduktion weiterhin visualisiert werden können.
Das Verfahren der PCA läuft nach folgendem Schema ab:

1. Berechnung des Mittelwerts und Zentrierung der Daten
2. Berechnung der Kovarianzmatrix
3. Berechnung der Eigenwerte und Eigenvektoren
4. Transformation der Daten

Damit die PCA korrekt funktioniert, muss zunächst von jeder Dimension der Mittelwert subtrahiert werden. Dieser Mittelwert entspricht dem Durchschnittswert jeder Dimension. Beispielsweise wird von allen $x$-Werten der Mittelwert $\overline{x}$ subtrahiert. Entsprechendes gilt für die anderen Dimensionen der Daten. Dadurch entsteht ein Datensatz mit einem Mittelwert von null.

Im nächsten Schritt wird die Kovarianzmatrix berechnet, welche die wechselseitigen Zusammenhänge zwischen den Merkmalen quantifiziert. Falls zwei Merkmale stark korrelieren, können diese in einer neuen Achse kombiniert werden.

Anschließend werden die Eigenwerte und Eigenvektoren der Kovarianzmatrix bestimmt. Die Eigenvektoren definieren die Richtungen der Hauptkomponenten, während die zugehörigen Eigenwerte die Bedeutung bzw. die Varianz der jeweiligen Eigenvektoren widerspiegeln.

Es folgt die eigentliche Dimensionsreduktion, indem nur diejenigen Eigenvektoren mit den größten Eigenwerten ausgewählt werden. Diese Eigenvektoren entsprechen den neuen Hauptachsen des Datensatzes.

Schließlich werden die Daten transformiert, indem die ursprüngliche Datenmatrix mit der Matrix der Eigenvektoren multipliziert wird. In dieser Matrix repräsentiert jede Spalte einen Eigenvektor.



In [ ]:
label_encoder = LabelEncoder()
categorical_columns = ['Geschlecht', 'EKG', 'AP']
for col in categorical_columns:
    data[col] = label_encoder.fit_transform(data[col])

In [ ]:
# Skalierung der Daten
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# PCA-Transformation mit zwei Hauptkomponenten
pca = PCA(n_components=2)
pca_result = pca.fit_transform(data_scaled)
df_pca = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])

# Interaktive Visualisierung
fig = px.scatter(df_pca, x='PC1', y='PC2', title='PCA-Visualisierung der Daten', opacity=0.5)
fig.show()

### Lässt sich aus den PCA-Daten eine potentielle gute Separierbarkeit der Klassen ablesen? 
TODO
--> Ich würde sagen nein, lass aber mal drüber quatschen 

## 3. Anwendung verschiedener vorgestellter Klassifikationsverfahren

#### Definition und Datenvorbereitung

In [6]:
# Kategorische und numerische Spalten definieren
categorical_features = ["Geschlecht", "EKG", "AP"]
numerical_features = ["Alter", "Blutdruck", "Chol", "Blutzucker", "HFmax", "RZ"]

# Zielvariable und Features auswählen
X = data[categorical_features + numerical_features]
y = data["KHK"]

# One-Hot-Encoding für kategorische Variablen
X = pd.get_dummies(X, columns=categorical_features)

# Standardisierung für numerische Variablen
scaler = StandardScaler()
X[numerical_features] = scaler.fit_transform(X[numerical_features])


# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 3.1 logistische Regression 

#### Modell definieren und trainieren

In [7]:
# Logistic Regression für binäre Klassifikation

# Pipeline mit Vorverarbeitung und logistische Regression
model = LogisticRegression()
# Modell trainieren
model.fit(X_train, y_train)

LogisticRegression()

#### Modell testen

In [8]:
# Vorhersagen treffen
y_pred_log_reg = model.predict(X_test)

# Evaluierung
accuracy = accuracy_score(y_test, y_pred_log_reg)
classification_rep = classification_report(y_test, y_pred_log_reg)

print(f"Accuracy: {accuracy:.2f}")
print(classification_rep)

Accuracy: 0.77
              precision    recall  f1-score   support

           0       0.70      0.78      0.74        77
           1       0.83      0.76      0.79       107

    accuracy                           0.77       184
   macro avg       0.76      0.77      0.76       184
weighted avg       0.77      0.77      0.77       184



### 3.2 Entscheidungsbäume

#### 3.2.1 klassische Entscheidungsbäume

In [9]:
clf_tree = DecisionTreeClassifier(random_state=42)
clf_tree.fit(X_train, y_train)

# Vorhersagen treffen
y_pred_tree = clf_tree.predict(X_test)

# Genauigkeit berechnen
accuracy_tree = accuracy_score(y_test, y_pred_tree)
classification_rep_tree = classification_report(y_test, y_pred_tree)
print(f"Modellgenauigkeit: {accuracy_tree:.2f}")
print(classification_rep_tree)

Modellgenauigkeit: 0.64
              precision    recall  f1-score   support

           0       0.56      0.64      0.59        77
           1       0.71      0.64      0.67       107

    accuracy                           0.64       184
   macro avg       0.63      0.64      0.63       184
weighted avg       0.64      0.64      0.64       184



#### 3.2.2 Bagging in Form von Random Forest

In [10]:
# Random Forest Modell trainieren
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Vorhersagen treffen
y_pred_random_forest = clf.predict(X_test)

# Genauigkeit berechnen
accuracy_random_forest = accuracy_score(y_test, y_pred_random_forest)
classification_rep = classification_report(y_test, y_pred_random_forest)
print(f"Modellgenauigkeit: {accuracy_random_forest:.2f}")
print(classification_rep)

Modellgenauigkeit: 0.75
              precision    recall  f1-score   support

           0       0.68      0.75      0.72        77
           1       0.81      0.75      0.78       107

    accuracy                           0.75       184
   macro avg       0.75      0.75      0.75       184
weighted avg       0.76      0.75      0.75       184



#### 3.2.3 Boosting in Form von AdaBoost

In [11]:
base_estimator = DecisionTreeClassifier(max_depth=1)
adaboost_model = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=50,
    learning_rate=0.3,
    random_state=42
)

adaboost_model.fit(X_train, y_train)

# Vorhersagen treffen
y_pred_ada = adaboost_model.predict(X_test)

# Genauigkeit berechnen
accuracy_random_forest = accuracy_score(y_test, y_pred_ada)
classification_rep = classification_report(y_test, y_pred_ada)
print(f"Modellgenauigkeit: {accuracy_random_forest:.2f}")
print(classification_rep)

Modellgenauigkeit: 0.77
              precision    recall  f1-score   support

           0       0.70      0.78      0.74        77
           1       0.83      0.76      0.79       107

    accuracy                           0.77       184
   macro avg       0.76      0.77      0.76       184
weighted avg       0.77      0.77      0.77       184



#### 3.2.4 Stacking

In [16]:
# Basismodelle: KNN, SVM und Logistische Regression
base_estimators = [
    ('knn', KNeighborsClassifier(n_neighbors=5)),  # KNN mit 5 Nachbarn
    ('svc', SVC(kernel='linear', random_state=42)),  # SVM mit linearem Kernel
    ('logreg', LogisticRegression(random_state=42))  # Logistische Regression
]

# Finales Modell (Meta-Modell)
final_estimator = LogisticRegression()

# StackingClassifier erstellen
stacking_model = StackingClassifier(estimators=base_estimators, final_estimator=final_estimator)

# Modell trainieren
stacking_model.fit(X_train, y_train)

# Vorhersagen treffen
y_pred_stack = stacking_model.predict(X_test)

# Genauigkeit berechnen
accuracy_stack = accuracy_score(y_test, y_pred_stack)
classification_rep = classification_report(y_test, y_pred_stack)

print(f"Modellgenauigkeit: {accuracy_stack:.2f}")
print(classification_rep)

Modellgenauigkeit: 0.76
              precision    recall  f1-score   support

           0       0.68      0.78      0.73        77
           1       0.82      0.74      0.78       107

    accuracy                           0.76       184
   macro avg       0.75      0.76      0.75       184
weighted avg       0.76      0.76      0.76       184



### 3.3 k-Nearest-Neighbor

#### 3.3.1 k-Nearest-Neighbor mit euklidischer Metrik

In [29]:
# k-NN Modell mit k=10 erstellen
knn_model = KNeighborsClassifier(n_neighbors=10, metric='euclidean')

# Modell trainieren
knn_model.fit(X_train, y_train)

# Vorhersagen treffen
y_pred_knn = knn_model.predict(X_test)

# Genauigkeit berechnen
accuracy_knn = accuracy_score(y_test, y_pred_knn)
classification_rep_knn = classification_report(y_test, y_pred_knn)

print(f"Modellgenauigkeit: {accuracy_knn:.2f}")
print(classification_rep_knn)


Modellgenauigkeit: 0.77
              precision    recall  f1-score   support

           0       0.68      0.83      0.75        77
           1       0.86      0.72      0.78       107

    accuracy                           0.77       184
   macro avg       0.77      0.78      0.77       184
weighted avg       0.78      0.77      0.77       184



#### 3.3.2 k-Nearest-Neighbor mit manhattan Metrik

In [30]:
# k-NN Modell mit k=10 erstellen
knn_model = KNeighborsClassifier(n_neighbors=10, metric='manhattan')

# Modell trainieren
knn_model.fit(X_train, y_train)

# Vorhersagen treffen
y_pred_knn = knn_model.predict(X_test)

# Genauigkeit berechnen
accuracy_knn = accuracy_score(y_test, y_pred_knn)
classification_rep_knn = classification_report(y_test, y_pred_knn)

print(f"Modellgenauigkeit: {accuracy_knn:.2f}")
print(classification_rep_knn)


Modellgenauigkeit: 0.74
              precision    recall  f1-score   support

           0       0.66      0.81      0.73        77
           1       0.83      0.70      0.76       107

    accuracy                           0.74       184
   macro avg       0.75      0.75      0.74       184
weighted avg       0.76      0.74      0.75       184



#### 3.3.4 k-Nearest-Neighbor mit Minkowski Metrik und p = 3

In [ ]:
# k-NN Modell mit k=10 erstellen
knn_model = KNeighborsClassifier(n_neighbors=10, metric='minkowski', p=3)

# Modell trainieren
knn_model.fit(X_train, y_train)

# Vorhersagen treffen
y_pred_knn = knn_model.predict(X_test)

# Genauigkeit berechnen
accuracy_knn = accuracy_score(y_test, y_pred_knn)
classification_rep_knn = classification_report(y_test, y_pred_knn)

print(f"Modellgenauigkeit: {accuracy_knn:.2f}")
print(classification_rep_knn)


Modellgenauigkeit: 0.74
              precision    recall  f1-score   support

           0       0.67      0.78      0.72        77
           1       0.82      0.72      0.77       107

    accuracy                           0.74       184
   macro avg       0.74      0.75      0.74       184
weighted avg       0.76      0.74      0.75       184



### 4. Support Vector Machine

In [34]:
# k-NN Modell mit k=10 erstellen
svm_model = SVC(kernel='linear', random_state=42)

# Modell trainieren
svm_model.fit(X_train, y_train)

# Vorhersagen treffen
y_pred_svm = svm_model.predict(X_test)

# Genauigkeit berechnen
accuracy_svm = accuracy_score(y_test, y_pred_svm)
classification_rep_svm = classification_report(y_test, y_pred_svm)

print(f"Modellgenauigkeit: {accuracy_svm:.2f}")
print(classification_rep_svm)


Modellgenauigkeit: 0.76
              precision    recall  f1-score   support

           0       0.69      0.78      0.73        77
           1       0.82      0.75      0.78       107

    accuracy                           0.76       184
   macro avg       0.76      0.76      0.76       184
weighted avg       0.77      0.76      0.76       184

